In [46]:
file_path = r"C:\Users\Shirley\Downloads\BGL\BGL.log"

with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
    for i in range(10):
        print(f.readline().strip())


- 1117838570 2005.06.03 R02-M1-N0-C:J12-U11 2005-06-03-15.42.50.363779 R02-M1-N0-C:J12-U11 RAS KERNEL INFO instruction cache parity error corrected
- 1117838570 2005.06.03 R02-M1-N0-C:J12-U11 2005-06-03-15.42.50.527847 R02-M1-N0-C:J12-U11 RAS KERNEL INFO instruction cache parity error corrected
- 1117838570 2005.06.03 R02-M1-N0-C:J12-U11 2005-06-03-15.42.50.675872 R02-M1-N0-C:J12-U11 RAS KERNEL INFO instruction cache parity error corrected
- 1117838570 2005.06.03 R02-M1-N0-C:J12-U11 2005-06-03-15.42.50.823719 R02-M1-N0-C:J12-U11 RAS KERNEL INFO instruction cache parity error corrected
- 1117838570 2005.06.03 R02-M1-N0-C:J12-U11 2005-06-03-15.42.50.982731 R02-M1-N0-C:J12-U11 RAS KERNEL INFO instruction cache parity error corrected
- 1117838571 2005.06.03 R02-M1-N0-C:J12-U11 2005-06-03-15.42.51.131467 R02-M1-N0-C:J12-U11 RAS KERNEL INFO instruction cache parity error corrected
- 1117838571 2005.06.03 R02-M1-N0-C:J12-U11 2005-06-03-15.42.51.293532 R02-M1-N0-C:J12-U11 RAS KERNEL INFO instr

In [3]:
pip uninstall logparser -y


Found existing installation: logparser 0.8.4
Uninstalling logparser-0.8.4:
  Successfully uninstalled logparser-0.8.4
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install git+https://github.com/logpai/logparser.git


In [2]:
from logparser import Drain


In [3]:
from logparser.Drain import LogParser
print("Drain parser loaded successfully!")


Drain parser loaded successfully!


In [4]:
import os

In [5]:
import re

In [38]:
# ===============================
# 🔧 Step 0 — Import dependencies
# ===============================
import re

# ===============================
# 📂 Step 1 — Input and Output paths
# ===============================
input_path = r"C:\Users\Shirley\Downloads\BGL\BGL.log"
output_path = r"C:\Users\Shirley\Downloads\BGL\BGL_clean_for_parsing.log"

# ===============================
# 🧹 Step 2 — Preprocess logs
# ===============================
total = 0
normal = 0
abnormal = 0

with open(input_path, "r", encoding="utf-8", errors="ignore") as infile, \
     open(output_path, "w", encoding="utf-8") as outfile:

    for line in infile:
        total += 1
        line = line.strip()
        if not line:
            continue  # skip empty

        parts = line.split()
        if len(parts) < 7:
            continue  # skip malformed lines

        # ✅ Step 2a: Label
        first_col = parts[0]
        if first_col == "-":
            label = 0
            normal += 1
        else:
            label = 1
            abnormal += 1

        # ✅ Step 2b: Timestamp
        timestamp = parts[4]  # e.g. 2005-06-03-15.42.50.363779

        # ✅ Step 2c: Message
        # Keep everything after the 5th field as message
        message = " ".join(parts[5:]).strip()

        # ✅ Output cleaned line
        outfile.write(f"{label}\t{timestamp}\t{message}\n")

print(f"✅ Preprocessing done.")
print(f"   Total lines processed: {total}")
print(f"   Normal (label=0): {normal}")
print(f"   Abnormal (label=1): {abnormal}")
print(f"   Saved cleaned file to: {output_path}")


✅ Preprocessing done.
   Total lines processed: 4747963
   Normal (label=0): 4399503
   Abnormal (label=1): 348460
   Saved cleaned file to: C:\Users\Shirley\Downloads\BGL\BGL_clean_for_parsing.log


In [39]:
import pandas as pd

df = pd.read_csv(r"C:\Users\Shirley\Downloads\BGL\BGL_clean_for_parsing.log",
                 sep="\t", names=["Label", "Timestamp", "Message"])

print("Total rows:", len(df))
print(df["Label"].value_counts())
print("Timestamp range:", df["Timestamp"].min(), "→", df["Timestamp"].max())
print(df.head(10))


Total rows: 4747963
Label
0    4399503
1     348460
Name: count, dtype: int64
Timestamp range: 2005-06-03-15.42.50.363779 → 2006-01-04-08.00.05.233639
   Label                   Timestamp  \
0      0  2005-06-03-15.42.50.363779   
1      0  2005-06-03-15.42.50.527847   
2      0  2005-06-03-15.42.50.675872   
3      0  2005-06-03-15.42.50.823719   
4      0  2005-06-03-15.42.50.982731   
5      0  2005-06-03-15.42.51.131467   
6      0  2005-06-03-15.42.51.293532   
7      0  2005-06-03-15.42.51.428563   
8      0  2005-06-03-15.42.51.601412   
9      0  2005-06-03-15.42.51.749199   

                                             Message  
0  R02-M1-N0-C:J12-U11 RAS KERNEL INFO instructio...  
1  R02-M1-N0-C:J12-U11 RAS KERNEL INFO instructio...  
2  R02-M1-N0-C:J12-U11 RAS KERNEL INFO instructio...  
3  R02-M1-N0-C:J12-U11 RAS KERNEL INFO instructio...  
4  R02-M1-N0-C:J12-U11 RAS KERNEL INFO instructio...  
5  R02-M1-N0-C:J12-U11 RAS KERNEL INFO instructio...  
6  R02-M1-N0-C:J12-U11 

In [40]:
import pandas as pd
from datetime import datetime

# ===============================
# 📂 Step 0 — Input file
# ===============================
file_path = r"C:\Users\Shirley\Downloads\BGL\BGL_clean_for_parsing.log"

# ===============================
# 🧾 Step 1 — Load preprocessed file
# ===============================
df = pd.read_csv(file_path, sep="\t", header=None, names=["Label", "Timestamp", "Message"])

# ===============================
# ✅ Step 2 — Basic checks
# ===============================
print("Total rows:", len(df))

# Label check
if set(df["Label"].unique()) <= {"0", "1"}:
    print("✅ Labels are valid (0/1)")
else:
    print("⚠️ Unexpected labels:", df["Label"].unique())

# Timestamp check
def is_valid_timestamp(ts):
    try:
        datetime.strptime(ts, "%Y-%m-%d-%H.%M.%S.%f")
        return True
    except:
        return False

invalid_ts = df[~df["Timestamp"].apply(is_valid_timestamp)]
if len(invalid_ts) == 0:
    print("✅ All timestamps are valid")
else:
    print(f"⚠️ {len(invalid_ts)} invalid timestamps found")

# Message check
empty_msg = df[df["Message"].str.strip() == ""]
if len(empty_msg) == 0:
    print("✅ All messages are non-empty")
else:
    print(f"⚠️ {len(empty_msg)} empty messages found")

# ===============================
# 📊 Step 3 — Additional statistics
# ===============================
print("\nLabel distribution:")
print(df["Label"].value_counts())

print("\nTimestamp range:")
print(df["Timestamp"].min(), "→", df["Timestamp"].max())

print("\nSample head:")
print(df.head(10).to_string(index=False))


Total rows: 4747963
⚠️ Unexpected labels: [0 1]
✅ All timestamps are valid
✅ All messages are non-empty

Label distribution:
Label
0    4399503
1     348460
Name: count, dtype: int64

Timestamp range:
2005-06-03-15.42.50.363779 → 2006-01-04-08.00.05.233639

Sample head:
 Label                  Timestamp                                                                      Message
     0 2005-06-03-15.42.50.363779 R02-M1-N0-C:J12-U11 RAS KERNEL INFO instruction cache parity error corrected
     0 2005-06-03-15.42.50.527847 R02-M1-N0-C:J12-U11 RAS KERNEL INFO instruction cache parity error corrected
     0 2005-06-03-15.42.50.675872 R02-M1-N0-C:J12-U11 RAS KERNEL INFO instruction cache parity error corrected
     0 2005-06-03-15.42.50.823719 R02-M1-N0-C:J12-U11 RAS KERNEL INFO instruction cache parity error corrected
     0 2005-06-03-15.42.50.982731 R02-M1-N0-C:J12-U11 RAS KERNEL INFO instruction cache parity error corrected
     0 2005-06-03-15.42.51.131467 R02-M1-N0-C:J12-U11 RAS KERNE

In [34]:
import pandas as pd
df = pd.read_csv(output_path, sep="\t", names=["Label", "Timestamp", "Message"])
print(df["Label"].unique())  # 会显示 [0] 表示全是正常
print(df["Label"].value_counts())


[0]
Label
0    4295303
Name: count, dtype: int64


In [41]:
with open(output_path, "r", encoding="utf-8") as f:
    for i in range(10):
        print(f.readline().strip())


0	2005-06-03-15.42.50.363779	R02-M1-N0-C:J12-U11 RAS KERNEL INFO instruction cache parity error corrected
0	2005-06-03-15.42.50.527847	R02-M1-N0-C:J12-U11 RAS KERNEL INFO instruction cache parity error corrected
0	2005-06-03-15.42.50.675872	R02-M1-N0-C:J12-U11 RAS KERNEL INFO instruction cache parity error corrected
0	2005-06-03-15.42.50.823719	R02-M1-N0-C:J12-U11 RAS KERNEL INFO instruction cache parity error corrected
0	2005-06-03-15.42.50.982731	R02-M1-N0-C:J12-U11 RAS KERNEL INFO instruction cache parity error corrected
0	2005-06-03-15.42.51.131467	R02-M1-N0-C:J12-U11 RAS KERNEL INFO instruction cache parity error corrected
0	2005-06-03-15.42.51.293532	R02-M1-N0-C:J12-U11 RAS KERNEL INFO instruction cache parity error corrected
0	2005-06-03-15.42.51.428563	R02-M1-N0-C:J12-U11 RAS KERNEL INFO instruction cache parity error corrected
0	2005-06-03-15.42.51.601412	R02-M1-N0-C:J12-U11 RAS KERNEL INFO instruction cache parity error corrected
0	2005-06-03-15.42.51.749199	R02-M1-N0-C:J12-U

In [42]:
# ===============================
# 🔧 Step 0 — Import dependencies
# ===============================
import os
import re
import hashlib
import pandas as pd
from tqdm import tqdm
from datetime import datetime

# ===============================
# 🧠 Step 1 — Define SimpleDrain Parser
# ===============================
class SimpleDrain:
    def __init__(self, st=0.5, max_templates=None, verbose=True):
        """
        st: similarity threshold (0~1), larger = stricter merging
        max_templates: optional limit on number of templates
        """
        self.st = st
        self.max_templates = max_templates
        self.templates = []
        self.template_counts = []
        self.log_to_template = []
        self.verbose = verbose

    def tokenize(self, line):
        """Split message into tokens"""
        return re.split(r"\s+", line.strip())

    def similarity(self, seq1, seq2):
        """Compute token-level similarity"""
        if not seq1 or not seq2:
            return 0.0
        same = sum(1 for i in range(min(len(seq1), len(seq2)))
                   if seq1[i] == seq2[i] or seq1[i] == "<*>" or seq2[i] == "<*>")
        return same / max(len(seq1), len(seq2))

    def merge_template(self, temp, tokens):
        """Merge two templates"""
        L = max(len(temp), len(tokens))
        merged = []
        for i in range(L):
            t = temp[i] if i < len(temp) else "<*>"
            s = tokens[i] if i < len(tokens) else "<*>"
            merged.append(t if t == s else "<*>")
        return merged

    def add_log(self, tokens):
        """Add new log to template pool"""
        best_sim, best_idx = -1.0, None
        for idx, temp in enumerate(self.templates):
            sim = self.similarity(temp, tokens)
            if sim > best_sim:
                best_sim, best_idx = sim, idx

        if best_sim >= self.st and best_idx is not None:
            new_temp = self.merge_template(self.templates[best_idx], tokens)
            self.templates[best_idx] = new_temp
            self.template_counts[best_idx] += 1
            return best_idx
        else:
            if self.max_templates and len(self.templates) >= self.max_templates:
                self.template_counts[best_idx] += 1
                new_temp = self.merge_template(self.templates[best_idx], tokens)
                self.templates[best_idx] = new_temp
                return best_idx
            else:
                self.templates.append(tokens)
                self.template_counts.append(1)
                return len(self.templates) - 1

    def parse_lines(self, lines):
        """Main parsing"""
        self.log_to_template = []
        iterator = tqdm(lines, desc="Parsing logs") if self.verbose else lines
        for line in iterator:
            tokens = self.tokenize(line)
            idx = self.add_log(tokens)
            self.log_to_template.append(idx)

    def save_results(self, lines, output_dir, base_name="BGL"):
        """Save structured results"""
        os.makedirs(output_dir, exist_ok=True)
        records = []
        for i, line in enumerate(lines):
            tidx = self.log_to_template[i]
            template_tokens = self.templates[tidx]
            template_str = " ".join(template_tokens)
            event_id = hashlib.md5(template_str.encode("utf-8")).hexdigest()[:8]
            records.append({
                "LineId": i + 1,
                "Content": line,
                "EventTemplate": template_str,
                "EventId": event_id
            })
        df_struct = pd.DataFrame(records)
        struct_path = os.path.join(output_dir, f"{base_name}_structured.csv")
        df_struct.to_csv(struct_path, index=False, encoding="utf-8")
        if self.verbose:
            print(f"✅ Saved structured logs: {struct_path}")

        # Template summary
        temps = []
        for i, temp in enumerate(self.templates):
            temp_str = " ".join(temp)
            event_id = hashlib.md5(temp_str.encode("utf-8")).hexdigest()[:8]
            occ = self.template_counts[i]
            temps.append({"EventId": event_id, "EventTemplate": temp_str, "Occurrences": occ})
        df_temp = pd.DataFrame(temps).sort_values(by="Occurrences", ascending=False)
        temp_path = os.path.join(output_dir, f"{base_name}_templates.csv")
        df_temp.to_csv(temp_path, index=False, encoding="utf-8")
        if self.verbose:
            print(f"✅ Saved template summary: {temp_path}")

# ===============================
# 📂 Step 2 — Paths
# ===============================
input_file = r"C:\Users\Shirley\Downloads\BGL\BGL_clean_for_parsing.log"
output_dir = r"C:\Users\Shirley\Downloads\BGL\result_new"
base_name = "BGL"

# ===============================
# 🔍 Step 3 — Load cleaned data
# ===============================
print("🔍 Loading cleaned log file...")
data = pd.read_csv(input_file, sep="\t", header=None, names=["Label", "Timestamp", "Message"], dtype=str)
print(f"✅ Loaded {len(data):,} log entries")

logs = data["Message"].tolist()

# ===============================
# ⚙️ Step 4 — Run SimpleDrain
# ===============================
parser = SimpleDrain(st=0.5, max_templates=None, verbose=True)
start_time = datetime.now()
parser.parse_lines(logs)
parser.save_results(logs, output_dir, base_name=base_name)
print(f"🕒 Total parsing time: {datetime.now() - start_time}")

# ===============================
# 🧾 Step 5 — Merge label + timestamp
# ===============================
structured_path = os.path.join(output_dir, f"{base_name}_structured.csv")
structured = pd.read_csv(structured_path)
structured["Label"] = data["Label"].astype(int).values
structured["Timestamp"] = data["Timestamp"].values

merged_path = os.path.join(output_dir, f"{base_name}_structured_labeled.csv")
structured.to_csv(merged_path, index=False, encoding="utf-8")
print(f"✅ Saved merged structured log: {merged_path}")

# ===============================
# 🔎 Step 6 — Quick sanity check
# ===============================
df_temp = pd.read_csv(os.path.join(output_dir, f"{base_name}_templates.csv"))
print("\nTop 10 templates:")
print(df_temp.head(10).to_string(index=False))

print("\nStructured log head:")
print(structured.head(10).to_string(index=False))
print(f"\nTotal structured rows: {len(structured):,}")


🔍 Loading cleaned log file...
✅ Loaded 4,747,963 log entries


Parsing logs: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4747963/4747963 [16:29<00:00, 4796.82it/s]


✅ Saved structured logs: C:\Users\Shirley\Downloads\BGL\result_new\BGL_structured.csv
✅ Saved template summary: C:\Users\Shirley\Downloads\BGL\result_new\BGL_templates.csv
🕒 Total parsing time: 0:17:55.183756
✅ Saved merged structured log: C:\Users\Shirley\Downloads\BGL\result_new\BGL_structured_labeled.csv

Top 10 templates:
 EventId                                                                                                                                                                                                                   EventTemplate  Occurrences
82e013db                                                                                                                                                                                         <*> <*> <*> <*> <*> <*> <*> <*> <*> <*>      1037941
f4efd1ca                                                                                                                                     <*> <*> <*> <*> <*> <*> <*> <*> <*> <*>

In [43]:
df = pd.read_csv(r"C:\Users\Shirley\Downloads\BGL\result_new\BGL_structured_labeled.csv")
print(df['Label'].value_counts())


Label
0    4399503
1     348460
Name: count, dtype: int64


In [44]:
# ============================================
# 🚀 Step 0 — Import dependencies
# ============================================
import os
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse
import numpy as np

# ============================================
# 📂 Step 1 — Input / Output paths
# ============================================
structured_path = r"C:\Users\Shirley\Downloads\BGL\result_new\BGL_structured_labeled.csv"
output_dir = r"C:\Users\Shirley\Downloads\BGL\result_new"
os.makedirs(output_dir, exist_ok=True)

tfidf_npz_path = os.path.join(output_dir, "BGL_tfidf_matrix.npz")
meta_csv_path = os.path.join(output_dir, "BGL_metadata.csv")

print("🔍 Loading structured log file...")
df = pd.read_csv(structured_path)
print(f"✅ Loaded {len(df):,} structured log entries")

# ============================================
# 🧠 Step 2 — Select text field for TF-IDF
# ============================================
# 如果 replication 需要按模板分析，使用 EventTemplate；
# 如果按原始日志分析，改为 df["Content"]
if "EventTemplate" in df.columns:
    texts = df["EventTemplate"].astype(str).values
else:
    texts = df["Content"].astype(str).values

# ============================================
# ⚙️ Step 3 — TF-IDF vectorization
# ============================================
print("🔧 Generating TF-IDF sparse matrix (max_features=5000)...")
vectorizer = TfidfVectorizer(max_features=5000, token_pattern=r"(?u)\b\w+\b")
tfidf_matrix = vectorizer.fit_transform(texts)

# ============================================
# 💾 Step 4 — Save sparse TF-IDF matrix + metadata
# ============================================
sparse.save_npz(tfidf_npz_path, tfidf_matrix)
meta_df = df[["Label", "Timestamp"]].copy()
meta_df.to_csv(meta_csv_path, index=False)

print(f"✅ TF-IDF sparse matrix saved to: {tfidf_npz_path}")
print(f"✅ Metadata (labels + timestamps) saved to: {meta_csv_path}")

# ============================================
# 🔎 Step 5 — Quick sanity check
# ============================================
print("\n🧩 Sanity check results:")
print(f"TF-IDF shape: {tfidf_matrix.shape}")
print(f"Non-zero entries: {tfidf_matrix.nnz:,} "
      f"({tfidf_matrix.nnz / (tfidf_matrix.shape[0] * tfidf_matrix.shape[1]):.4%} density)")
print(f"Label distribution:\n{meta_df['Label'].value_counts()}")
print(f"Timestamp range: {meta_df['Timestamp'].iloc[0]} → {meta_df['Timestamp'].iloc[-1]}")

# Verify alignment
if len(meta_df) == tfidf_matrix.shape[0]:
    print("✅ Row count matches between TF-IDF matrix and metadata.")
else:
    print("❌ Row count mismatch — check preprocessing or parsing outputs.")


🔍 Loading structured log file...
✅ Loaded 4,747,963 structured log entries
🔧 Generating TF-IDF sparse matrix (max_features=5000)...
✅ TF-IDF sparse matrix saved to: C:\Users\Shirley\Downloads\BGL\result_new\BGL_tfidf_matrix.npz
✅ Metadata (labels + timestamps) saved to: C:\Users\Shirley\Downloads\BGL\result_new\BGL_metadata.csv

🧩 Sanity check results:
TF-IDF shape: (4747963, 144)
Non-zero entries: 4,195,276 (0.6136% density)
Label distribution:
Label
0    4399503
1     348460
Name: count, dtype: int64
Timestamp range: 2005-06-03-15.42.50.363779 → 2006-01-04-08.00.05.233639
✅ Row count matches between TF-IDF matrix and metadata.


In [45]:
import os
bgl_dir = r"C:\Users\Shirley\Downloads\BGL"

print("Files in BGL folder:")
for f in os.listdir(bgl_dir):
    print("-", f)


Files in BGL folder:
- anaconda_projects
- BGL.log
- BGL_clean.log
- BGL_clean_for_parsing.log
- result_custom
- result_full
- result_new


In [39]:
import os
notebook_path = os.path.abspath("bgl_analysis.ipynb")
print(notebook_path)


C:\Windows\System32\bgl_analysis.ipynb


In [2]:
import os
os.path.abspath("")


'C:\\Windows\\System32'